<a href="https://colab.research.google.com/github/giumanuz/AoC22/blob/main/Tesi_SATELLITI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torchvision.transforms as tr
from skimage import io
from torchvision.transforms.functional import InterpolationMode

# Models
# from unet import Unet
# from siamunet_conc import SiamUnet_conc
# from siamunet_diff import SiamUnet_diff
# from fresunet import FresUNet

# Other
import os
import numpy as np
import random
# from skimage import io
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
# %matplotlib inline
from tqdm import tqdm as tqdm
from pandas import read_csv
from math import floor, ceil, sqrt, exp
from IPython import display
import time
from itertools import chain
import time
import warnings
from pprint import pprint

import random

# Function for setting the seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

set_seed(42)

print('IMPORTS OK')

IMPORTS OK


In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
# Global Variables' Definitions

PATH_TO_DATASET = '/content/drive/MyDrive/Start'
PATH_TO_LABELS = '/content/drive/MyDrive/Finish'

BATCH_SIZE = 64
PATCH_SIDE = 96
N_EPOCHS = 50

NORMALISE_IMGS = True

LOAD_TRAINED = False

DATA_AUG = True


print('DEFINITIONS OK')

DEFINITIONS OK


In [115]:
def get_tif_images(tif_path):
    images = []
    for filename in os.listdir(tif_path):
        if filename.endswith('.tif'):
            images.append(filename)
    return images

print(get_tif_images(PATH_TO_DATASET))


class MyDataset(Dataset):
    """
    Custom class to load the dataset
    """
    def __init__(self, tif_path, transform=None):
        """
        Args:
            tif_path (string): path to tif files
            transform (callable, optional): Optional transform to be applied on a sample
        """
        # read tif files from tif_path directory
        self.transform = transform
        self.names = get_tif_images(tif_path)


        self.data = torch.zeros(len(self.names), 3, PATCH_SIDE, PATCH_SIDE)
        self.labels = torch.zeros(len(self.names), 1, PATCH_SIDE, PATCH_SIDE, dtype=torch.long)

        for idx, name in enumerate(self.names):
            img_name = os.path.join(tif_path, name)
            image = io.imread(img_name)
            label_name = os.path.join(PATH_TO_LABELS, name[:-9] + '_label.tif')
            label = io.imread(label_name)
            if self.transform:
                image = self.transform(image)
                label = self.transform(label)

            label = self.convert_image(label)
            self.data[idx] = image
            self.labels[idx] = label


    def __len__(self):
        return len(self.data)

    def closest_to_one(self, num):
        if abs(num - 1) < abs(num - 0):
            return 1
        else:
            return 0


    def convert_image(self, tensor):
        """
        Convert tensor from 3 channel to one channel, and associate it a number from 0 to 5
        """
        # Crea una mappa dei colori
        color_map = {}

        # transform image in tensor
        # tensor = torch.tensor(image)

        # print(tensor.shape)
        
        for x in range(tensor.size(1)):
            for y in range(tensor.size(2)):
                pixel_value = tensor[:, x, y]




                # pixel_value=pixel_value.tolist()
                # for idx, elem in enumerate(pixel_value):
                #     pixel_value[idx] = self.closest_to_one(elem)
                # pixel_value = torch.tensor(pixel_value)




                if not torch.equal(pixel_value, torch.tensor([0, 0, 0])):
                    if tuple(pixel_value.tolist()) not in color_map:
                        color_map[tuple(pixel_value.tolist())] = len(color_map)
        
        converted_tensor = torch.zeros(1, tensor.size(1), tensor.size(2), dtype=torch.long)
    
        for x in range(tensor.size(1)):
            for y in range(tensor.size(2)):
                pixel_value = tensor[:, x, y]

                
                color = color_map.get(tuple(pixel_value.tolist()), 0)
                
                # Imposta il valore del pixel nel nuovo tensore
                converted_tensor[0, x, y] = color      #modify
        
        # Stampa il tensore convertito
        # print("Tensore convertito:")
        # print(converted_tensor)


        counts = torch.bincount(converted_tensor.flatten(), minlength=6)
        for key, value in color_map.items():
            print(f"{value}: {key} --- n_volte: {counts[value]}")



        return converted_tensor




    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

['top_potsdam_2_10_IRRG.tif', 'top_potsdam_2_11_IRRG.tif', 'top_potsdam_2_12_IRRG.tif', 'top_potsdam_2_13_IRRG.tif', 'top_potsdam_2_14_IRRG.tif', 'top_potsdam_3_10_IRRG.tif', 'top_potsdam_3_11_IRRG.tif', 'top_potsdam_3_12_IRRG.tif', 'top_potsdam_3_13_IRRG.tif', 'top_potsdam_3_14_IRRG.tif', 'top_potsdam_4_10_IRRG.tif', 'top_potsdam_4_11_IRRG.tif', 'top_potsdam_4_12_IRRG.tif', 'top_potsdam_4_13_IRRG.tif', 'top_potsdam_4_14_IRRG.tif', 'top_potsdam_4_15_IRRG.tif', 'top_potsdam_5_10_IRRG.tif', 'top_potsdam_5_11_IRRG.tif', 'top_potsdam_5_12_IRRG.tif', 'top_potsdam_5_13_IRRG.tif', 'top_potsdam_5_14_IRRG.tif', 'top_potsdam_5_15_IRRG.tif', 'top_potsdam_6_7_IRRG.tif', 'top_potsdam_6_8_IRRG.tif', 'top_potsdam_6_9_IRRG.tif', 'top_potsdam_6_10_IRRG.tif', 'top_potsdam_6_11_IRRG.tif', 'top_potsdam_6_12_IRRG.tif', 'top_potsdam_6_13_IRRG.tif', 'top_potsdam_6_14_IRRG.tif', 'top_potsdam_6_15_IRRG.tif', 'top_potsdam_7_7_IRRG.tif', 'top_potsdam_7_8_IRRG.tif', 'top_potsdam_7_9_IRRG.tif', 'top_potsdam_7_10_I

In [116]:
if DATA_AUG:
    data_transform = tr.Compose([
        tr.ToPILImage(),
        tr.RandomHorizontalFlip(), 
        tr.RandomVerticalFlip(), 
        tr.RandomRotation(180),
        tr.Resize((PATCH_SIDE, PATCH_SIDE), interpolation=InterpolationMode.NEAREST),
        tr.ToTensor(),
    ])
else:
    data_transform = None


dataset = MyDataset(tif_path=PATH_TO_DATASET, transform=data_transform)

0: (0, 1, 0) --- n_volte: 2068
1: (0, 1, 1) --- n_volte: 3349
2: (1, 1, 1) --- n_volte: 1416
3: (0, 0, 1) --- n_volte: 1090
4: (1, 0, 0) --- n_volte: 1244
5: (1, 1, 0) --- n_volte: 49
0: (1, 1, 1) --- n_volte: 2909
1: (1, 0, 0) --- n_volte: 697
2: (0, 1, 1) --- n_volte: 2977
3: (0, 1, 0) --- n_volte: 1820
4: (1, 1, 0) --- n_volte: 78
5: (0, 0, 1) --- n_volte: 735
0: (0, 0, 1) --- n_volte: 2198
1: (0, 1, 1) --- n_volte: 3997
2: (1, 1, 1) --- n_volte: 1404
3: (0, 1, 0) --- n_volte: 1508
4: (1, 1, 0) --- n_volte: 71
5: (1, 0, 0) --- n_volte: 38
0: (1, 1, 1) --- n_volte: 2374
1: (0, 1, 0) --- n_volte: 2133
2: (0, 1, 1) --- n_volte: 2724
3: (1, 0, 0) --- n_volte: 167
4: (0, 0, 1) --- n_volte: 1710
5: (1, 1, 0) --- n_volte: 108
0: (0, 1, 1) --- n_volte: 4948
1: (1, 1, 1) --- n_volte: 1191
2: (0, 1, 0) --- n_volte: 2622
3: (1, 0, 0) --- n_volte: 22
4: (0, 0, 1) --- n_volte: 375
5: (1, 1, 0) --- n_volte: 58
0: (0, 0, 1) --- n_volte: 2309
1: (1, 1, 1) --- n_volte: 1920
2: (0, 1, 0) --- n_volte:

KeyboardInterrupt: ignored

In [ ]:
print(dataset.names[12])

In [ ]:
if NORMALISE_IMGS:
    mean = torch.mean(dataset.data, dim=(0, 2, 3), keepdim=True)
    std = torch.std(dataset.data, dim=(0, 2, 3), keepdim=True) 

    print(f'mean: {mean}')
    print(f'std: {std}')

    print(dataset.data.shape)

    dataset.data = (dataset.data - mean) / std

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)


print('DATASET OK')

In [ ]:
print(next(iter(train_dataloader))[0].shape)

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF


class DoubleConvolution(nn.Module):

    def __init__(self, in_ch, out_ch):
        super(DoubleConvolution, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class UNet(nn.Module):

    def __init__(self, input_channels=3, output_channels=1,
                 features=[64, 128, 256, 512]):
        super(UNet, self).__init__()

        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Encoder 
        for f in features:
            self.downs.append(DoubleConvolution(input_channels, f))
            input_channels = f

        # lower bottleneck layers
        self.bottleneck = DoubleConvolution(features[-1], features[-1] * 2)

        # Decoder
        for f in reversed(features):
            self.ups.append(
                nn.Sequential(
                    nn.Upsample(scale_factor=2),
                    nn.Conv2d(in_channels=2 * f, out_channels=f, kernel_size=3,
                              padding=1),
                ))
            self.ups.append(DoubleConvolution(2 * f, f))

        self.final_convolution = nn.Conv2d(in_channels=features[0],
                                           out_channels=output_channels,
                                           kernel_size=3, padding=1)

    def forward(self, x):
        skip_connections = list()
        for module in self.downs:
            x = module(x)
            skip_connections.append(x)
            x = self.pool(x)

        skip_connections = skip_connections[::-1]  # reverse order

        x = self.bottleneck(x)

        for i in range(0, len(self.ups), 2):
            x = self.ups[i](x)
            skip_connection = skip_connections[i // 2]
            if skip_connection.shape != x.shape:
                x = TF.resize(x, size=skip_connection.shape[2:],
                              interpolation=TF.InterpolationMode.NEAREST)
            x = torch.cat([skip_connection, x], dim=1)
            x = self.ups[i + 1](x)

        x = self.final_convolution(x)

        return x

In [ ]:
# train
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device='cuda'):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs).to(device)
            loss = loss_fn(output, targets.squeeze(1))  # modify
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
            print("ciao")
        training_loss /= len(train_loader.dataset)
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output, targets.squeeze(1))  # modify
            valid_loss += loss.data.item() * inputs.size(0)
        valid_loss /= len(val_loader.dataset)
        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch, training_loss, valid_loss))

# test
def test(model, test_loader, device='cuda'):
    model.eval()
    test_loss = 0.0
    num_correct = 0 
    num_examples = 0
    for batch in test_loader:
        inputs, targets = batch
        inputs = inputs.to(device)
        output = model(inputs)
        targets = targets.to(device)
        loss = loss_fn(output, targets)
        test_loss += loss.data.item() * inputs.size(0)
    test_loss /= len(test_loader.dataset)
    print('Test Loss: {:.4f}'.format(test_loss))

# create model



model = UNet(input_channels=3, output_channels=6)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

class_counts = torch.bincount(dataset.labels.flatten())
print(class_counts)
     
class_weights = 1.0 / class_counts.float()
class_weights /= torch.sum(class_weights)
class_weights = class_weights.to(device)  # Move class_weights to CUDA device

print(class_weights)

loss_fn = nn.CrossEntropyLoss(weight=class_weights.to(device))

# train
train(model, optimizer, loss_fn, train_dataloader, test_dataloader, epochs=N_EPOCHS, device=device)

# test
test(model, test_dataloader, device=device)


In [ ]:
# test
test(model, test_dataloader, device=device)

In [ ]:
import matplotlib.pyplot as plt

# Valori dell'epoch e training loss
epochs = list(range(50))
training_loss = [1.6720, 1.6712, 1.6714, 1.5992, 1.6178, 1.6141, 1.6306, 1.6682, 1.6405, 1.6349, 1.6153, 1.6410, 1.6804, 1.5805, 1.6081, 1.6171, 1.6613, 1.6400, 1.6406, 1.6639, 1.6242, 1.6031, 1.6117, 1.6504, 1.6335, 1.5848, 1.6550, 1.5941, 1.6115, 1.6011, 1.5929, 1.6480, 1.6041, 1.6028, 1.5579, 1.6334, 1.6339, 1.6265, 1.6217, 1.6433, 1.5718, 1.6148, 1.6083, 1.6162, 1.6040, 1.6548, 1.6762, 1.6452, 1.6120, 1.5993]

# Valore del test loss
test_loss = 1.6232

# Plot del training loss
plt.plot(epochs, training_loss, label='Training Loss')

# Linea per il test loss
plt.axhline(y=test_loss, color='r', linestyle='--', label='Test Loss')

# Etichette degli assi
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Titolo del grafico
plt.title('Training Loss and Test Loss')

# Legenda
plt.legend()

# Mostra il grafico
plt.show()
